# PART 1: PHPMYADMIN (queries are copy pasted on here)

In [ ]:
# Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT 
    name
FROM 
    Facilities
WHERE 
    membercost > 0

In [ ]:
Q2: How many facilities do not charge a fee to members? */

SELECT 
    COUNT(*)
FROM 
    Facilities
WHERE 
    membercost = 0

In [ ]:
Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. 

SELECT 
    facid,name,membercost,monthlymaintenance
FROM 
    Facilities
WHERE 
    membercost > 0 
	AND membercost < (0.2 * monthlymaintenance)

In [ ]:
Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT * 
FROM 
    Facilities
WHERE 
    facid IN (1,5)

In [ ]:
Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT 
    name,
    monthlymaintenance,
    CASE WHEN monthlymaintenance <= 100
        THEN 'cheap'
        ELSE 'expensive'
        END AS label 
FROM 
    Facilities

In [ ]:
Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT 
    firstname,
    surname
FROM 
    Members
WHERE 
    joindate = (SELECT MAX(joindate) 
                  FROM Members)

In [ ]:
Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT 
    DISTINCT CONCAT(m.firstname,' ',m.surname) AS member_name, 
    f.name as facility_name
FROM 
    Bookings as b
LEFT JOIN 
    Facilities AS f ON b.facid = f.facid
LEFT JOIN 
    Members AS m ON b.memid = m.memid
WHERE 
    f.name LIKE 'Tennis%'
ORDER BY 
    member_name

In [ ]:
Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT 
    DISTINCT CONCAT(m.firstname,' ',m.surname) AS member_name, 
    f.name as facility_name,
    CASE WHEN b.memid = 0
        THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots
        END AS cost
FROM 
    Bookings as b
LEFT JOIN 
    Facilities AS f ON b.facid = f.facid
LEFT JOIN 
    Members AS m ON b.memid = m.memid
WHERE 
    b.starttime LIKE '2012-09-14%'
    AND CASE WHEN b.memid = 0
            THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots 
            END
        < 30
ORDER BY 
    cost DESC

In [ ]:
Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT 
    DISTINCT CONCAT(m.firstname,' ',m.surname) AS member_name, 
    f.name as facility_name,
    CASE WHEN b.memid = 0
        THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots
        END AS cost
FROM 
    Bookings as b
LEFT JOIN 
    Facilities AS f ON b.facid = f.facid
LEFT JOIN 
    Members AS m ON b.memid = m.memid
WHERE 
    b.starttime LIKE '2012-09-14%' 
    AND bookid IN (SELECT b.bookid 
	FROM Bookings as b 
	LEFT JOIN Facilities AS f ON b.facid = f.facid
	WHERE  (b.memid =0 AND f.guestcost * b.slots >30) OR 			(b.memid >0 AND f.membercost * b.slots >30))
ORDER BY cost DESC

# PART 2: SQLite

In [8]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "/Users/sharanaravindh/Desktop/springboard/Github repository/SQLFiles Tier 1/sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [13]:
def select_revenue_summary(conn):
    
    cur = conn.cursor()
    
    query = """
        WITH c AS (
            SELECT
                f.facid,
                CASE
                    WHEN b.memid = 0 THEN f.guestcost * b.slots
                    ELSE f.membercost * b.slots
                END AS cost
            FROM
                Facilities as f
            LEFT JOIN
                Bookings AS b ON f.facid = b.facid
        )
        SELECT
            f.name,
            SUM(c.cost) AS total_revenue
        FROM
            Facilities as f
        INNER JOIN
            c ON f.facid = c.facid
        GROUP BY
            f.facid, f.name
        HAVING
            SUM(c.cost) < 1000
        ORDER BY
            total_revenue;
        """
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "/Users/sharanaravindh/Desktop/springboard/Github repository/SQLFiles Tier 1/sqlite_db_pythonsqlite.db"
 
    conn = create_connection(database)
    with conn: 
        print("Facilities with total revenue less than 1000:")
        select_revenue_summary(conn)
 
if __name__ == '__main__':
    main()

2.6.0
Facilities with total revenue less than 1000:
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [15]:
def member_and_recommender(conn):
    """
    Query members and their recommenders' names
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query = """
    WITH x AS (
        SELECT
            m.recommendedby,
            CASE
                WHEN m.recommendedby = 0 THEN 'None'
                ELSE r.surname || ',' || r.firstname
            END AS recommender_name
        FROM
            Members AS m
        LEFT JOIN
            Members AS r ON m.recommendedby = r.memid
    )
    SELECT DISTINCT
        m.surname || ',' || m.firstname AS member_name,
        x.recommender_name
    FROM
        Members AS m
    LEFT JOIN
        x ON m.recommendedby = x.recommendedby
    ORDER BY
        member_name;
    """
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "/Users/sharanaravindh/Desktop/springboard/Github repository/SQLFiles Tier 1/sqlite_db_pythonsqlite.db"
 
    conn = create_connection(database)
    with conn: 
        print("Member and Recommender")
        member_and_recommender(conn)
 
if __name__ == '__main__':
    main()

2.6.0
Member and Recommender
('Bader,Florence', 'Stibbons,Ponder')
('Baker,Anne', 'Stibbons,Ponder')
('Baker,Timothy', 'Farrell,Jemima')
('Boothe,Tim', 'Rownam,Tim')
('Butters,Gerald', 'Smith,Darren')
('Coplin,Joan', 'Baker,Timothy')
('Crumpet,Erica', 'Smith,Tracy')
('Dare,Nancy', 'Joplette,Janice')
('Farrell,David', None)
('Farrell,Jemima', None)
('GUEST,GUEST', None)
('Genting,Matthew', 'Butters,Gerald')
('Hunt,John', 'Purview,Millicent')
('Jones,David', 'Joplette,Janice')
('Jones,Douglas', 'Jones,David')
('Joplette,Janice', 'Smith,Darren')
('Mackenzie,Anna', 'Smith,Darren')
('Owen,Charles', 'Smith,Darren')
('Pinker,David', 'Farrell,Jemima')
('Purview,Millicent', 'Smith,Tracy')
('Rownam,Tim', None)
('Rumney,Henrietta', 'Genting,Matthew')
('Sarwin,Ramnaresh', 'Bader,Florence')
('Smith,Darren', None)
('Smith,Jack', 'Smith,Darren')
('Smith,Tracy', None)
('Stibbons,Ponder', 'Tracy,Burton')
('Tracy,Burton', None)
('Tupperware,Hyacinth', None)
('Worthington-Smyth,Henry', 'Smith,Tracy')


In [16]:
def facility_usage_by_members(conn):
    """
    Query facilities and their usage by members (excluding guests)
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query = """
    SELECT
        f.name AS facility_name,
        COUNT(b.bookid) AS member_usage
    FROM
        Facilities AS f
    JOIN
        Bookings AS b ON f.facid = b.facid
    WHERE
        b.memid <> 0
    GROUP BY
        f.name
    ORDER BY
        facility_name;
    """
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "/Users/sharanaravindh/Desktop/springboard/Github repository/SQLFiles Tier 1/sqlite_db_pythonsqlite.db"
 
    conn = create_connection(database)
    with conn: 
        print("Facility Usage by Members")
        facility_usage_by_members(conn)
 
if __name__ == '__main__':
    main()

2.6.0
Facility Usage by Members
('Badminton Court', 344)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Pool Table', 783)
('Snooker Table', 421)
('Squash Court', 195)
('Table Tennis', 385)
('Tennis Court 1', 308)
('Tennis Court 2', 276)


In [17]:
def facility_usage_by_month(conn):
    """
    Query facilities and their usage by month (excluding guests)
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query = """
    SELECT
        strftime('%Y-%m', b.starttime) AS year_month,
        f.name AS facility_name,
        COUNT(b.bookid) AS facility_usage
    FROM
        Facilities AS f
    JOIN
        Bookings AS b ON f.facid = b.facid
    WHERE
        b.memid <> 0
    GROUP BY
        year_month, facility_name
    ORDER BY
        year_month, facility_name;
    """
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "/Users/sharanaravindh/Desktop/springboard/Github repository/SQLFiles Tier 1/sqlite_db_pythonsqlite.db"
 
    conn = create_connection(database)
    with conn: 
        print("Facility Usage by Month")
        facility_usage_by_month(conn)
 
if __name__ == '__main__':
    main()

2.6.0
Facility Usage by Month
('2012-07', 'Badminton Court', 51)
('2012-07', 'Massage Room 1', 77)
('2012-07', 'Massage Room 2', 4)
('2012-07', 'Pool Table', 103)
('2012-07', 'Snooker Table', 68)
('2012-07', 'Squash Court', 23)
('2012-07', 'Table Tennis', 48)
('2012-07', 'Tennis Court 1', 65)
('2012-07', 'Tennis Court 2', 41)
('2012-08', 'Badminton Court', 132)
('2012-08', 'Massage Room 1', 153)
('2012-08', 'Massage Room 2', 9)
('2012-08', 'Pool Table', 272)
('2012-08', 'Snooker Table', 154)
('2012-08', 'Squash Court', 85)
('2012-08', 'Table Tennis', 143)
('2012-08', 'Tennis Court 1', 111)
('2012-08', 'Tennis Court 2', 109)
('2012-09', 'Badminton Court', 161)
('2012-09', 'Massage Room 1', 191)
('2012-09', 'Massage Room 2', 14)
('2012-09', 'Pool Table', 408)
('2012-09', 'Snooker Table', 199)
('2012-09', 'Squash Court', 87)
('2012-09', 'Table Tennis', 194)
('2012-09', 'Tennis Court 1', 132)
('2012-09', 'Tennis Court 2', 126)
